In [14]:
import mxnet as mx
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

sample_count = 1000
train_count = 800
valid_count = sample_count - train_count

feature_count = 100
category_count = 10
batch=10

In [2]:
X = mx.nd.uniform(low=0, high=1, shape=(sample_count,feature_count))

print(X.shape)
print(X.asnumpy())

(1000, 100)
[[ 0.54881352  0.59284461  0.71518934 ...,  0.60783064  0.36371076
   0.32504722]
 [ 0.57019675  0.03842543  0.43860152 ...,  0.36980811  0.00469548
   0.14644176]
 [ 0.67781651  0.5696184   0.27000797 ...,  0.46924916  0.99884701
   0.84114015]
 ..., 
 [ 0.59305507  0.41538411  0.24317271 ...,  0.58364183  0.8284471
   0.4361586 ]
 [ 0.71750212  0.5193429   0.80764365 ...,  0.15779524  0.86435819
   0.72405148]
 [ 0.82241368  0.72197711  0.85116214 ...,  0.24351381  0.55131787
   0.12545156]]


In [3]:
Y = mx.nd.empty((sample_count,))
for i in range(0,sample_count-1):
  Y[i] = np.random.randint(0,category_count)

print(Y.shape)
print(Y[0:10].asnumpy())

(1000,)
[ 0.  1.  1.  4.  8.  4.  4.  0.  3.  2.]


In [5]:
X_train = mx.nd.crop(X, begin=(0,0), end=(train_count,feature_count))
print(X_train.shape)

X_valid = mx.nd.crop(X, begin=(train_count,0), end=(sample_count,feature_count))
print(X_valid.shape)

Y_train = Y[0:train_count]
print(Y_train.shape)

Y_valid = Y[train_count:sample_count]
print(Y_valid.shape)

(800, 100)
(200, 100)
(800,)
(200,)


In [6]:
data = mx.sym.Variable('data')
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=64)
relu1 = mx.sym.Activation(fc1, name='relu1', act_type="relu")
fc2 = mx.sym.FullyConnected(relu1, name='fc2', num_hidden=category_count)
out = mx.sym.SoftmaxOutput(fc2, name='softmax')
mod = mx.mod.Module(out)

In [15]:
train_iter = mx.io.NDArrayIter(data=X_train,label=Y_train,batch_size=batch)

In [16]:
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))
mod.fit(train_iter, num_epoch=60)

/Users/taemyung/anaconda/envs/py35/lib/python3.5/site-packages/mxnet/module/base_module.py:463: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.111250
INFO:root:Epoch[0] Time cost=0.047
INFO:root:Epoch[1] Train-accuracy=0.147500
INFO:root:Epoch[1] Time cost=0.048
INFO:root:Epoch[2] Train-accuracy=0.167500
INFO:root:Epoch[2] Time cost=0.044
INFO:root:Epoch[3] Train-accuracy=0.185000
INFO:root:Epoch[3] Time cost=0.043
INFO:root:Epoch[4] Train-accuracy=0.217500
INFO:root:Epoch[4] Time cost=0.044
INFO:root:Epoch[5] Train-accuracy=0.230000
INFO:root:Epoch[5] Time cost=0.045
INFO:root:Epoch[6] Train-accuracy=0.251250
INFO:root:Epoch[6] Time cost=0.043
INFO:root:Epoch[7] Train-accuracy=0.263750
INFO:root:Epoch[7] Time cost=0.043
INFO:root:Epoch[8] Train-accuracy=0.287500
INFO:root:Epoch[8] Time cost=0.043
INFO:root:Epoch[9] Train-accuracy=0.313750
INFO:root:Epo

In [17]:
pred_iter = mx.io.NDArrayIter(data=X_valid,label=Y_valid, batch_size=batch)
pred_count = valid_count

In [19]:
correct_preds = total_correct_preds = 0
print('batch [labels] [predicted labels]  correct predictions')

for preds, i_batch, batch in mod.iter_predict(pred_iter):
    label = batch.label[0].asnumpy().astype(int)
    pred_label = preds[0].asnumpy().argmax(axis=1)
    correct_preds = np.sum(pred_label==label)
    print(i_batch, label, pred_label, correct_preds)
    total_correct_preds = total_correct_preds + correct_preds

print('Validation accuracy: %2.2f' % (1.0*total_correct_preds/pred_count))

batch [labels] [predicted labels]  correct predictions
0 [5 8 5 5 2 6 6 0 7 8] [2 7 4 2 7 4 5 3 7 4] 1
1 [3 5 7 5 7 9 8 5 9 6] [5 7 5 6 9 4 9 5 5 3] 1
2 [5 5 4 1 1 6 6 2 8 7] [7 1 7 9 2 1 1 6 5 4] 0
3 [9 7 0 6 8 5 7 5 1 1] [3 1 2 7 6 7 2 1 4 3] 0
4 [5 4 2 4 0 7 4 0 9 3] [1 5 2 7 2 5 7 7 6 2] 1
5 [2 4 9 4 0 3 8 2 2 7] [8 1 9 5 1 7 8 5 2 8] 3
6 [8 0 5 5 6 1 1 8 2 5] [1 7 9 7 6 4 2 7 4 5] 2
7 [3 3 5 4 2 9 2 3 1 2] [5 0 3 3 7 1 6 5 8 8] 0
8 [2 4 1 2 6 5 8 9 0 4] [6 4 5 4 4 4 8 0 2 5] 2
9 [1 9 1 7 4 7 8 5 8 0] [4 1 1 5 3 3 1 2 5 2] 1
10 [9 2 5 1 8 8 4 7 8 0] [5 3 5 4 4 9 7 3 6 5] 1
11 [6 8 7 7 2 7 2 3 2 5] [1 5 2 2 5 9 2 9 6 3] 1
12 [5 0 3 1 5 0 5 9 2 9] [1 1 5 7 5 9 5 5 5 2] 2
13 [1 9 3 9 0 4 5 6 6 3] [6 7 7 5 4 3 5 5 8 1] 1
14 [6 4 1 8 5 2 1 6 3 5] [9 5 3 9 7 3 7 5 5 3] 0
15 [0 9 6 4 8 7 9 9 2 3] [9 6 5 7 1 8 4 1 9 7] 0
16 [6 6 3 6 5 4 3 6 3 6] [9 7 7 2 2 5 1 3 5 2] 0
17 [0 2 1 0 1 5 6 3 1 6] [5 8 5 1 1 9 2 3 2 5] 2
18 [4 3 2 1 2 7 7 1 5 5] [8 5 3 9 8 1 4 4 8 2] 0
19 [7 6 5 7 1 6 5 0 6 0]